In [2]:
import pandas as pd
import tensorflow as tf  # Importa o TensorFlow para construção de modelos de aprendizado de máquina
import sklearn
import scikeras

In [1]:
from scikeras.wrappers import KerasClassifier  # Importa KerasClassifier do scikeras para usar com o modelo Keras
from tensorflow.keras import backend as k  # Importa o backend Keras para gerenciamento de sessões
from tensorflow.keras.models import Sequential  # Importa a classe Sequential para criar uma rede neural sequencial
from sklearn.model_selection import GridSearchCV  # Importa a classe GridSearchCV para otimização de hiperparâmetros


In [4]:
X = pd.read_csv('entradas_breast.csv')
y = pd.read_csv('saidas_breast.csv')

In [30]:
def criar_rede(optimizer, loss, kernel_initializer, activation, neurons):
    """
    Cria uma rede neural sequencial para classificação binária com parâmetros configuráveis.
    
    Args:
        optimizer: Otimizador a ser utilizado durante o treinamento do modelo.
        loss: Função de perda a ser utilizada na compilação do modelo.
        kernel_initializer: Inicializador dos pesos das camadas da rede.
        activation: Função de ativação a ser utilizada nas camadas ocultas.
        neurons (int): Número de unidades nas camadas ocultas.

    Returns:
        Sequential: A rede neural configurada.
    """
    # Limpa qualquer sessão anterior do Keras para evitar conflitos com modelos anteriores
    tf.keras.backend.clear_session()
    
    # Cria uma nova rede neural sequencial
    rede_neural = Sequential([
        # Camada de entrada que recebe dados com 30 características
        tf.keras.layers.InputLayer(shape=(30,)),  # Define a forma da entrada com 30 características
        
        # Camada oculta: camada densa com 'neurons' neurônios e função de ativação especificada
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        
        # Camada de saída: camada densa com 1 neurônio para previsão binária (0 ou 1)
        tf.keras.layers.Dense(units=1, activation='sigmoid')  # Usando sigmoide para obter uma probabilidade
    ])
    
    # Compila o modelo da rede neural, definindo a função de perda, o otimizador e as métricas de desempenho
    rede_neural.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])  # Configura a perda e o otimizador
    
    return rede_neural  # Retorna a rede neural criada

In [42]:
parametros = {
    'batch_size': [10],  # Mantém o tamanho de lote
    'model__kernel_initializer': ['he_uniform', 'glorot_uniform'],
    'epochs': [50],  # Número reduzido de épocas para treinamento
    'model__optimizer': ['adam'],  # Mantém apenas o otimizador Adam
    'model__loss': ['binary_crossentropy'],  # Função de perda para teste
    'model__activation': ['relu'],  # Mantém apenas a função de ativação ReLU
    'model__neurons': [8],  # Mantém um número menor de unidades nas camadas ocultas
}

In [43]:
# Cria uma instância de KerasClassifier
rede_neural = KerasClassifier(model=criar_rede)

In [44]:
parametros

{'batch_size': [10],
 'model__kernel_initializer': ['he_uniform', 'glorot_uniform'],
 'epochs': [50],
 'model__optimizer': ['adam'],
 'model__loss': ['binary_crossentropy'],
 'model__activation': ['relu'],
 'model__neurons': [8]}

In [45]:
# Cria uma instância de GridSearchCV para encontrar a melhor combinação de hiperparâmetros
grid_search = GridSearchCV(estimator=rede_neural, param_grid=parametros,
                           scoring='accuracy', cv=5)

In [46]:
# Fit do GridSearchCV nos dados de treinamento
grid_search = grid_search.fit(X, y)  # X e y devem ser definidos anteriormente como os dados de treinamento2

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 755us/step - binary_accuracy: 0.4298 - loss: 232.9958 
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - binary_accuracy: 0.4473 - loss: 91.8843
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - binary_accuracy: 0.4697 - loss: 30.2572
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - binary_accuracy: 0.5600 - loss: 19.7305
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - binary_accuracy: 0.5744 - loss: 17.3097
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - binary_accuracy: 0.5831 - loss: 15.2267
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - binary_accuracy: 0.5929 - loss: 12.5956
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - binary_accuracy: 0.6212 - loss: 11.2984
Epoch 9/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - binary_accuracy: 0.6301 - loss: 7.0519
Epoch 10/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - binary_accuracy: 0.6346 - loss: 6.4717
Epoch 11/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - bina

In [47]:
        # Obtém os melhores parâmetros encontrados pelo GridSearchCV
melhores_parametros = grid_search.best_params_

# Obtém o melhor score (pontuação) médio encontrado pelo GridSearchCV durante a validação cruzada
melhor_score = grid_search.best_score_





In [48]:
# Imprime os melhores parâmetros encontrados
print(melhores_parametros)


{'batch_size': 10, 'epochs': 50, 'model__activation': 'relu', 'model__kernel_initializer': 'he_uniform', 'model__loss': 'binary_crossentropy', 'model__neurons': 8, 'model__optimizer': 'adam'}


In [49]:
# Imprime a melhor pontuação média encontrada
print(melhor_score)


0.8294364229156963


In [50]:

# Imprime o objeto grid_search para detalhes adicionais, incluindo todos os resultados da busca
print(grid_search)

GridSearchCV(cv=5,
             estimator=KerasClassifier(model=<function criar_rede at 0x00000219C7F351C0>),
             param_grid={'batch_size': [10], 'epochs': [50],
                         'model__activation': ['relu'],
                         'model__kernel_initializer': ['he_uniform',
                                                       'glorot_uniform'],
                         'model__loss': ['binary_crossentropy'],
                         'model__neurons': [8], 'model__optimizer': ['adam']},
             scoring='accuracy')
